## 任务1：模型训练与预测

- 步骤1：导入LightGBM库
- 步骤2：使用LGBMClassifier对iris进行训练。
- 步骤3：将预测的模型对iris进行预测

In [37]:
import lightgbm as lgb
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score,auc
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [60]:
iris = datasets.load_iris()
X = pd.DataFrame(data = iris.data , columns=iris.feature_names)
y = pd.DataFrame(iris.target,columns=['target'])

data = pd.concat([X,y],axis=1)
data.head()

array([0, 1, 2])

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2022)


In [5]:
## sklearn API

lgb_model = lgb.LGBMClassifier(objective= 'multiclass',
                               metric = 'multi_logloss',
                               boosting_type= 'gbdt',
                               n_estimators= 100,
                               learning_rate= 0.2,
                               n_jobs= -1,
                               random_state= 2022
                               )


lgb_model.fit(X_train,y_train)
y_pre = lgb_model.predict(X_test)
print('预测值：{}'.format(y_pre))
print('模型f1值：{}'.format(f1_score(y_test,y_pre,average='macro')))

预测值：[2 2 0 2 0 0 1 1 0 1 1 2 1 2 2 0 1 1 2 1 0 0 2 0 2 2 2 0 1 2]
模型f1值：0.9670588235294119


In [103]:
## 原生API

params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class':3, # 多分类，需指定类别数
    'metric': 'multi_logloss',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}


lgb_train = lgb.Dataset(X_train,y_train)
lgb_val = lgb.Dataset(X_test,y_test,reference=lgb_train)


gbm = lgb.train(params,
          lgb_train,
          num_boost_round=100,
          valid_sets=[lgb_train, lgb_val]
          )

# predict ,输出的是每个类别的概率
y_pre = gbm.predict(X_test)
print(y_pre) 

# 输出类别 (这里是输出最大值的index)
y_pre = [list(x).index(max(x)) for x in y_pre]
print('预测值：{}'.format(y_pre))
print(accuracy_score(y_test,y_pre))

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000367 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	training's multi_logloss: 1.02645	valid_1's multi_logloss: 1.04308
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	training's multi_logloss: 0.963391	valid_1's multi_logloss: 0.981793
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light

## 任务2：模型保存与加载

https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/advanced_example.py
- 步骤1：将任务1训练得到的模型，使用pickle进行保存。
- 步骤2：将任务1训练得到的模型，使用json进行保存。
- 步骤3：加载步骤1和步骤2的模型，并进行预测。

### pickle

In [25]:
# pickle 保存模型

import pickle
def pkl_save(filename,file):
    output = open(filename, 'wb')
    pickle.dump(file, output)
    output.close()

def pkl_load(filename):
    pkl_file = open(filename, 'rb')
    file = pickle.load(pkl_file) 
    pkl_file.close()
    return file

## 模型保存
pkl_save('./lgb_model.pkl',lgb_model)

## 加载模型
lgb_pkl_load = pkl_load('./lgb_model.pkl')

## 预测
lgb_pkl_load.predict(X_test)

array([2, 2, 0, 2, 0, 0, 1, 1, 0, 1, 1, 2, 1, 2, 2, 0, 1, 1, 2, 1, 0, 0,
       2, 0, 2, 2, 2, 0, 1, 2])

### joblib

In [31]:
import joblib
# sklearn.externals.joblib被弃用了

## 模型保存
joblib.dump(lgb_model,'lgb_model.pkl')

## 加载模型
lgb_joblib_load = joblib.load('./lgb_model.pkl')

## 预测
lgb_joblib_load.predict(X_test)

array([2, 2, 0, 2, 0, 0, 1, 1, 0, 1, 1, 2, 1, 2, 2, 0, 1, 1, 2, 1, 0, 0,
       2, 0, 2, 2, 2, 0, 1, 2])

### json

Pickle 和 Joblib 库简单快捷，易于使用，但是在不同的 Python 版本之间存在兼容性问题，且不同模型也有所不同。json的兼容性较好。

In [101]:
import json

# 保存模型
model_json = gbm.dump_model()
with open('model.json', 'w+') as f:
    json.dump(model_json, f, indent=4)

# 加载模型
with open('model.json', 'r', encoding="UTF-8") as f:
		gbm = json.load(f)

## 任务3：分类、回归和排序任务

https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/sklearn_example.py
https://github.com/microsoft/LightGBM/blob/master/examples/python-guide/simple_example.py

- 步骤1：学习LightGBM中sklearn接口的使用，导入分类、回归和排序模型。
- 步骤2：学习LightGBM中原生train接口的使用。
- 步骤3：二分类任务
    - 使用make_classification，创建一个二分类数据集。
    - 使用sklearn接口完成训练和预测。
    - 使用原生train接口完成训练和预测。
- 步骤4：多分类任务
    - 使用make_classification，创建一个多分类数据集。
    - 使用sklearn接口完成训练和预测。
    - 使用原生train接口完成训练和预测。
- 步骤5：回归任务
    - 使用make_regression，创建一个回归数据集。
    - 使用sklearn接口完成训练和预测。
    - 使用原生train接口完成训练和预测。